In [1]:
%load_ext autoreload
%autoreload 2
import sys
paths = ['../models', '../dataset', '../util']
for path in paths:
    if path not in sys.path:
        sys.path.append(path)

import json
from keras.models import Sequential, load_model
from keras.layers import Convolution2D, Dense, Flatten, Dropout, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import np_utils
from collections import defaultdict, Counter
import cv2

import pandas as pd
import matplotlib.pyplot as plt
import glob
import numpy as np
import os
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib

from scipy.misc import imread

import dataset
from data_generators import get_data
from bounding_boxes import get_bounding_boxes, largest_bbox_per_image

%matplotlib inline

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release.  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 960M (CNMeM is disabled, cuDNN 5110)


In [2]:
DATA_FOLDER = 'C:/Users/Thomas/Documents/MLIP-BigAITuna/Data'
#bboxes = get_bounding_boxes('/media/joris/Scrub/Data/bounding_boxes')
#bboxes = largest_bbox_per_image(bboxes)
bboxes = {}
files = glob.glob(DATA_FOLDER + '/bounding_boxes/*.json')
for ff in files:
    with open(ff) as f:
        data = json.load(f)
        for image in data:
            for annot in image['annotations']:
                bboxes[os.path.basename(image['filename'])] = (int(max(0, annot['x'])), int(max(0, annot['y'])), int(annot['width']), int(annot['height'])) 
train_data = dataset.train
val_data = dataset.validation
labels = dataset.labels

In [3]:
train_data = train_data[train_data.filename.isin(bboxes.keys())]
val_data = val_data[val_data.filename.isin(bboxes.keys())]
test_files = sorted([os.path.basename(p) for p in glob.glob(os.path.join(DATA_FOLDER, 'test_stg1/*.jpg'))])
test_data = pd.DataFrame({'filename':test_files, 'label': ['ALB']*len(test_files)})

print len(train_data)
print len(val_data)

2654
656


In [4]:
model = Sequential()
model.add(BatchNormalization(input_shape=[3, 256, 256], axis=1))
model.add(Convolution2D(16, 3, 3, border_mode='same', input_shape=[3, 256, 256], 
                        activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(32, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(len(labels), activation='softmax'))

model.compile(loss='categorical_crossentropy',
             optimizer=Adam(),
             metrics=['accuracy', 'categorical_crossentropy'])

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_1 (BatchNorma (None, 3, 256, 256)   12          batchnormalization_input_1[0][0] 
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 16, 256, 256)  448         batchnormalization_1[0][0]       
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 16, 128, 128)  0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 32, 128, 128)  4640        maxpooling2d_1[0][0]             
___________________________________________________________________________________________

In [5]:
train_gen = get_data(train_data, os.path.join(DATA_FOLDER, 'train'), augmentation=True,
                     bboxes=bboxes, flip=True, shift_x=5, shift_y=5, rot_range=10, 
                     elastic_trans=False)
val_gen = get_data(val_data, os.path.join(DATA_FOLDER, 'train'), bboxes=bboxes,
                   augmentation=False, shuffle=False)


In [ ]:
batch = next(train_gen)
print batch[1]
# for img in batch[0]:
#     print img.shape
#     plt.imshow(img.transpose(1, 2, 0) + 0.5)
#     plt.show()

[[0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1 0]
 [0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]]


In [ ]:
csv_logger = CSVLogger('run4_adam.csv')
lr_plateau = ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1, factor=0.5)
checkpoint = ModelCheckpoint(filepath='../model/model.{epoch:02d}-{val_loss}.hdf5',
                             verbose=1, save_best_only=True)

model.fit_generator(train_gen, samples_per_epoch=len(train_data),
                    nb_epoch=500,
                    callbacks=[csv_logger, lr_plateau, checkpoint],
                    validation_data=val_gen, nb_val_samples=len(val_data),
                    pickle_safe=False, nb_worker=3)

D:\Anaconda2\lib\site-packages\keras\models.py:907: UserWarning: The "nb_worker" argument is deprecated when pickle_safe is False
  warnings.warn('The "nb_worker" argument is deprecated '


Epoch 1/500
2624/2654 [============================>.] - ETA: 112s - loss: 2.0928 - acc: 0.1562 - categorical_crossentropy: 2.0928 - ETA: 103s - loss: 4.9246 - acc: 0.1562 - categorical_crossentropy: 4.9246 - ETA: 100s - loss: 5.7136 - acc: 0.1250 - categorical_crossentropy: 5.7136 - ETA: 96s - loss: 5.3815 - acc: 0.1328 - categorical_crossentropy: 5.3815  - ETA: 95s - loss: 4.8273 - acc: 0.1375 - categorical_crossentropy: 4.8273

D:\Anaconda2\lib\site-packages\keras\engine\training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


Epoch 00000: val_loss improved from inf to 1.67212, saving model to ../model/model.00-1.67211525213.hdf5
2656/2654 [==============================] - 123s - loss: 1.8935 - acc: 0.3238 - categorical_crossentropy: 1.8935 - val_loss: 1.6721 - val_acc: 0.4435 - val_categorical_crossentropy: 1.6721
Epoch 2/500
2656/2654 [==============================] - 117s - loss: 1.1149 - acc: 0.6020 - categorical_crossentropy: 1.1149 - val_loss: 1.1302 - val_acc: 0.6652 - val_categorical_crossentropy: 1.1302
Epoch 3/500
2656/2654 [==============================] - 126s - loss: 0.8254 - acc: 0.7029 - categorical_crossentropy: 0.8254 - val_loss: 0.8764 - val_acc: 0.6979 - val_categorical_crossentropy: 0.8764
Epoch 4/500
2656/2654 [==============================]

In [11]:
model = load_model('/media/joris/Scrub/Models/fcn_crop/model.19-0.585225485265.hdf5')

In [15]:
def remove_labels(gen):
    for x, _ in gen:
        yield x
test_gen = get_data(test_data, os.path.join(DATA_FOLDER, 'test'), bboxes=bboxes,
                   augmentation=False, shuffle=False)
test_gen = remove_labels(test_gen)

In [16]:
preds = model.predict_generator(test_gen, val_samples=1024)[:1000]

In [21]:
preds = np.clip(preds, 0.02, 0.98)

In [22]:
with open('sub.csv', 'w') as sub_file:
    sub_file.write('image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT\n')
    for img, pred in zip(test_files, preds):
        csv = '%s,%s\n' % (img, ','.join([str(f) for f in pred]))
        sub_file.write(csv)